# Stateful model with Keras - long-term dependencies understanding

In [1]:
from keras.models import Sequential
from keras.layers import Embedding, Activation, LSTM, Dense, TimeDistributed, Dropout, BatchNormalization
from keras.utils import *

Using TensorFlow backend.


In [2]:
import numpy as np
from numpy.random import choice

In [5]:
import requests

In [6]:
url = 'http://lib.ru/LITRA/PUSHKIN/p2.txt'
get_url = requests.get(url)
text = get_url.text

In [7]:
text[1180:2280]

'\n\nПТИЧКА\n\nВ чужбине свято наблюдаю\nРодной обычай старины:\nНа волю птичку выпускаю\nПри светлом празднике весны.\n\nЯ стал доступен утешенью;\nЗа что на бога мне роптать,\nКогда хоть одному творенью\nЯ мог свободу даровать!\n\n\nЦАРСКОЕ СЕЛО\n\nХранитель милых чувств и прошлых наслаждений,\nО ты, певцу дубрав давно знакомый гений,\nВоспоминание, рисуй передо мной\nВолшебные места, где я живу душой,\nЛеса, где я любил, где чувство развивалось,\nГде с первой юностью младенчество сливалось\nИ где, взлелеянный природой и мечтой,\nЯ знал поэзию, веселость и покой...\n\nВеди, веди меня под липовые сени,\nВсегда любезные моей свободной лени,\nНа берег озера, на тихий скат холмов!..\nДа вновь увижу я ковры густых лугов,\nИ дряхлый пук дерев, и светлую долину,\nИ злачных берегов знакомую картину,\nИ в тихом озере, средь блещущих зыбей,\nСтаницу гордую спокойных лебедей.\n\n\n* * *\n\nКто, волны, вас остановил,\nКто оковал ваш бег могучий,\nКто в пруд безмолвный и дремучий\nПоток мятежный 

In [8]:
print('Corpus length: ', len(text))

Corpus length:  576904


In [9]:
chars = sorted(list(set(text)))

In [10]:
''.join(chars[:])

'\n !"&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz{}АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюя'

In [11]:
vocab_size = len(chars)
print("Vocab size: ", vocab_size)

Vocab size:  151


In [31]:
char_ind = dict((c,i) for i,c in enumerate(chars))
#char_ind = {}
#for i,c in enumerate(chars):
#    char_ind[c]=i

In [28]:
print('Char indices: ', char_ind)

Char indeces:  {'\n': 0, ' ': 1, '!': 2, '"': 3, '&': 4, "'": 5, '(': 6, ')': 7, '*': 8, '+': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '<': 26, '=': 27, '>': 28, '?': 29, '@': 30, 'A': 31, 'B': 32, 'C': 33, 'D': 34, 'E': 35, 'F': 36, 'G': 37, 'H': 38, 'I': 39, 'J': 40, 'K': 41, 'L': 42, 'M': 43, 'N': 44, 'O': 45, 'P': 46, 'Q': 47, 'R': 48, 'S': 49, 'T': 50, 'U': 51, 'V': 52, 'W': 53, 'X': 54, 'Y': 55, 'Z': 56, '[': 57, ']': 58, '_': 59, 'a': 60, 'b': 61, 'c': 62, 'd': 63, 'e': 64, 'f': 65, 'g': 66, 'h': 67, 'i': 68, 'j': 69, 'k': 70, 'l': 71, 'm': 72, 'n': 73, 'o': 74, 'p': 75, 'q': 76, 'r': 77, 's': 78, 't': 79, 'u': 80, 'v': 81, 'w': 82, 'x': 83, 'y': 84, 'z': 85, '{': 86, '}': 87, 'А': 88, 'Б': 89, 'В': 90, 'Г': 91, 'Д': 92, 'Е': 93, 'Ж': 94, 'З': 95, 'И': 96, 'Й': 97, 'К': 98, 'Л': 99, 'М': 100, 'Н': 101, 'О': 102, 'П': 103, 'Р': 104, 'С': 105, 'Т': 106, 'У': 107, 'Ф': 108, 'Х'

In [29]:
ind_char = dict((i,c) for i,c in enumerate(chars))
#char_ind = {}
#for i,c in enumerate(chars):
#    char_ind[i]=c

In [22]:
print('Indices of chars: ', ind_char)

Indeces of chars:  {0: '\n', 1: ' ', 2: '!', 3: '"', 4: '&', 5: "'", 6: '(', 7: ')', 8: '*', 9: '+', 10: ',', 11: '-', 12: '.', 13: '/', 14: '0', 15: '1', 16: '2', 17: '3', 18: '4', 19: '5', 20: '6', 21: '7', 22: '8', 23: '9', 24: ':', 25: ';', 26: '<', 27: '=', 28: '>', 29: '?', 30: '@', 31: 'A', 32: 'B', 33: 'C', 34: 'D', 35: 'E', 36: 'F', 37: 'G', 38: 'H', 39: 'I', 40: 'J', 41: 'K', 42: 'L', 43: 'M', 44: 'N', 45: 'O', 46: 'P', 47: 'Q', 48: 'R', 49: 'S', 50: 'T', 51: 'U', 52: 'V', 53: 'W', 54: 'X', 55: 'Y', 56: 'Z', 57: '[', 58: ']', 59: '_', 60: 'a', 61: 'b', 62: 'c', 63: 'd', 64: 'e', 65: 'f', 66: 'g', 67: 'h', 68: 'i', 69: 'j', 70: 'k', 71: 'l', 72: 'm', 73: 'n', 74: 'o', 75: 'p', 76: 'q', 77: 'r', 78: 's', 79: 't', 80: 'u', 81: 'v', 82: 'w', 83: 'x', 84: 'y', 85: 'z', 86: '{', 87: '}', 88: 'А', 89: 'Б', 90: 'В', 91: 'Г', 92: 'Д', 93: 'Е', 94: 'Ж', 95: 'З', 96: 'И', 97: 'Й', 98: 'К', 99: 'Л', 100: 'М', 101: 'Н', 102: 'О', 103: 'П', 104: 'Р', 105: 'С', 106: 'Т', 107: 'У', 108: 'Ф',

In [35]:
idx = [char_ind[element] for element in text]
#idx = []
#for element in text:
#    element = char_ind[element]
#    idx.append(element)

In [36]:
idx[:5]

[26, 67, 79, 72, 71]

In [41]:
max_len = 40
sentences = [idx[elem:elem+max_len] for elem in range(len(idx)-max_len+1)]
#sentences = []
#for elem in range(0, len(idx)-max_len+1):
#    sentences.append(idx[elem: elem+max_len])
next_char = [idx[elem+1:elem+max_len+1] for elem in range(len(idx)-max_len+1)]
#next_char = []
#for elem in range(0, len(idx)-max_len+1):
#    next_char.append(elem[item+1: elem+max_len+1])
print("Sequence of indices: ", sentences[:2])
print("Sequence of indices with train element: ", next_char[:2])
print('Number sentences: ', len(sentences))
print('Number next char sentences: ', len(next_char))

Sequence of indices:  [[26, 67, 79, 72, 71, 28, 26, 67, 64, 60, 63, 28, 26, 79, 68, 79, 71, 64, 28, 88, 130, 124, 129, 136, 119, 132, 123, 135, 1, 105, 124, 135, 122, 124, 124, 121, 127, 142, 1, 103], [67, 79, 72, 71, 28, 26, 67, 64, 60, 63, 28, 26, 79, 68, 79, 71, 64, 28, 88, 130, 124, 129, 136, 119, 132, 123, 135, 1, 105, 124, 135, 122, 124, 124, 121, 127, 142, 1, 103, 138]]
Sequence of indices with train element:  [[67, 79, 72, 71, 28, 26, 67, 64, 60, 63, 28, 26, 79, 68, 79, 71, 64, 28, 88, 130, 124, 129, 136, 119, 132, 123, 135, 1, 105, 124, 135, 122, 124, 124, 121, 127, 142, 1, 103, 138], [79, 72, 71, 28, 26, 67, 64, 60, 63, 28, 26, 79, 68, 79, 71, 64, 28, 88, 130, 124, 129, 136, 119, 132, 123, 135, 1, 105, 124, 135, 122, 124, 124, 121, 127, 142, 1, 103, 138, 143]]
Number sentences:  576865
Number next char sentences:  576865


In [60]:
sentences = np.concatenate([[np.array(elem)] for elem in sentences[:-2]])
#np_sentences = np.array([sentences[0]])
#for elem in sentences[1:-2]:
#    elem = np.array([elem])
#    np_sentences = np.concatenate((np_sentences, elem), axis=0)
next_char = np.concatenate([[np.array(elem)] for elem in next_char[:-2]])
print("Array of lists in sentences: ", sentences[:2])
print('Array of lists in next char: ', next_char[:2])
sentences.shape, next_char.shape

Array of lists in sentences:  [[ 26  67  79  72  71  28  26  67  64  60  63  28  26  79  68  79  71  64
   28  88 130 124 129 136 119 132 123 135   1 105 124 135 122 124 124 121
  127 142   1 103]
 [ 67  79  72  71  28  26  67  64  60  63  28  26  79  68  79  71  64  28
   88 130 124 129 136 119 132 123 135   1 105 124 135 122 124 124 121 127
  142   1 103 138]]
Array of lists in next char:  [[ 67  79  72  71  28  26  67  64  60  63  28  26  79  68  79  71  64  28
   88 130 124 129 136 119 132 123 135   1 105 124 135 122 124 124 121 127
  142   1 103 138]
 [ 79  72  71  28  26  67  64  60  63  28  26  79  68  79  71  64  28  88
  130 124 129 136 119 132 123 135   1 105 124 135 122 124 124 121 127 142
    1 103 138 143]]


((576857, 40), (576855, 40))

In [61]:
n_fac = 24

In [62]:
model = Sequential([
    Embedding(vocab_size, n_fac, input_length=max_len), #vocab_size is a amount of indices, and indices themselves of course (vertical column of embedding matrix)
                                                        #thus vocab_size related to unique char indices in our example
                                                        #n_fac is a amount of integers (vector) for every index in vocab_sze (horizontal line of embedding matrix) 
    BatchNormalization(), #gives well scaled inputs
    LSTM(512, return_sequences=True), #return_sequences= says return output every time-step rather than returning single output at the end
    Dropout(0.2),
    TimeDistributed(Dense(vocab_size)), #TimeDistributed() towards layer in it (Dense layer in this example) 
                                        #creates copies of that layer, which amount is same as dimension of that layer 
                                        #every copy will share same weight matrix (vocab_size in this example)
                                        #i.e. in Keras if there is return_sequences=True - any Dense layer after that will have to have TD

    Activation('softmax')    
])

In [63]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 40, 24)            3624      
_________________________________________________________________
batch_normalization_2 (Batch (None, 40, 24)            96        
_________________________________________________________________
lstm_2 (LSTM)                (None, 40, 512)           1099776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 40, 512)           0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 40, 151)           77463     
_________________________________________________________________
activation_2 (Activation)    (None, 40, 151)           0         
Total params: 1,180,959
Trainable params: 1,180,911
Non-trainable params: 48
_________________________________________________________________

In [64]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [87]:
model.fit(sentences, np.expand_dims(next_char, -1), batch_size=64, epochs=1)

Epoch 1/1
576863/576863 [==============================] - 42608s 74ms/step - loss: 1.7514


In [50]:
def print_example_2(seed_string):
    for i in range(420): #iteration makes prediction character by character #range(value) is a lengh of predicting corpus 
        x = np.array([char_ind[c] for c in seed_string[-40:]])[np.newaxis,:] #converting seed_string into array of indices
        #x = np.array([[char_ind[seed_string[-40]]]]) #[-40] because prediction adding one char to corpus, shifting needed
        #for c in seed_string[-39:]: #iteration except char, which is already in array
        #    c = np.array([[char_ind[c]]])
        #    x = np.concatenate((x, c), axis=1)
        
        preds=model.predict(x,verbose=0)[0][-1] #[0][-1] because we predicting next char after [-1] value #verbosity mode, 0 or 1. 
        preds=preds/np.sum(preds) #Array-wise sum devided by element in array #function?
        #preds are probabilities associated with each char in shars (amount of 151 floats related to every unique char in chars)
        
        next_char=np.random.choice(chars,p=preds) #Generates a random sample from chars array 
                                                  #p:probabilities associated with each entry
        #basically random should find best probablility, when with its index we can find its related char in chars dictionary  
        #counter=-1
        #for elem in np.nditer(preds):
        #    counter+=1
        #    if elem == np.amax(preds):  #or np.argmax()
        #        break
        #next_char = chars[counter]
        
        seed_string=seed_string+next_char #Simply concatenate seed_string
    print(seed_string)

In [100]:
print_example_2('Навис покров угрюмой нощи \nНа своде дрем')

Навис покров угрюмой нощи 
На своде дремает Байрона
Судна того любил он по мнему;
Журналист мне светленный,
Тем грози церковь участие вершины
Лавороски, подарат как перевет.
Как он волк, как син. Он вместе в черновик соловей дворянин.

Не фарнала в тейра, чки тоще
     Да будут  тачен пред ним торкую
И к найдушною излованье
Воюны за длянет начала крестелить мой орман "прозил недвижим, по мыслей подаром.


* * *
Первая мать из, сиды в нем обестели!
А сам за


In [98]:
model.save_weights('C:/Users/Gavrilov/My projects/LSTM_weights_2.h5')

In [109]:
print_example_2('Уж я не тот любовник страстный,\n Кому ди')

Уж я не тот любовник страстный,
 Кому диститета роман в Кому книжей заимственный луча,
Мласницую мою дорожу
Все отвечает подтиром
Разбегал в меня робзянье...
Клянусь видно, слава  своих,
общество, сердце своей праздновался.
Неизъясняемой послушной,
Ты помнишь ли, о нет не помянеть разона,
И само чужой рощу Ику?
       И выставил в Бастник.


НА ФОСКОМУ

Подрежа мнет бренных является взды.. Захладном долгой новой.

Он чужкавые сердце моей.

Стихи внимал в к


In [119]:
print_example_2('Слыхали ль вы за рощей глас ночной\n Певца')

Слыхали ль вы за рощей глас ночной
 Певца едушенной".
Без обиды, погляд, удалит,
Что ты в скалы небесный,
Твой долго  на  письма
Пушкина, писал о доходе при Павлихо и дане;
В мюня, даль его родных
листо дни лежалось поздное недурный и привычей.


* * *

Был и внесть а могилой,
Готов вольных меня
Не могил глубоком,
Тихо любизноваты, други.
Благослови.

Мой гора и веселья,
Покитель твой дорогой
Была, чья мудливо и сил же дабого,
Серебра на ала ветли жизни пес
